In [ ]:
import itertools
import os
import sys

import numpy as np
import pandas as pd
from numba import njit
from plotly.subplots import make_subplots
from vectorbt.generic import nb as generic_nb
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import vectorbt as vbt
from lib.utils import file_to_data_frame, LR, ExtendedPortfolio, plot_series_vs_scatters, dropnaninf, get_best_index, \
    where_true_set_series

In [ ]:
file = "/Users/pilo/development/itba/pf/Binance_Minute_OHLC_CSVs/3000/Binance_ADAUSDT_minute_3000.csv"
_, ohlcv = file_to_data_frame(file)
ohlcv. head()

In [ ]:
close = ohlcv["Close"]
volume = ohlcv["Volume"]
lr_ind = LR.run(close)
print(lr_ind.lr.shape)

In [ ]:
@njit
def signal_calculations(lr, lr_ma, lr_mstd, vol, vol_mstd, lr_thld, vol_thld):
    lr_thld_std = lr_thld * lr_mstd
    ups = lr_ma - lr_thld_std # lr_thld < 0
    downs = lr_ma + lr_thld_std
    lr_entries = np.where(lr < downs, True, False)
    lr_exits = np.where(lr > ups, True, False)
    vol_thld_std = vol_thld * vol_mstd
    vol_entries = np.where(vol > vol_thld_std, True, False)
    return ups, downs, lr_entries, lr_exits, vol_thld_std, vol_entries

@njit
def ma_mstd(lr, volume, lag):
    lr_ma = generic_nb.rolling_mean_nb(lr, lag)
    lr_mstd = generic_nb.rolling_std_nb(lr, lag)
    vol_mstd = generic_nb.rolling_std_nb(volume, lag)
    return lr_ma, lr_mstd, vol_mstd

@njit
def signals_nb(lr, volume, lr_thld, vol_thld, lag):
    lr_ma, lr_mstd, vol_mstd = ma_mstd(lr, volume, lag)
    _, _, lr_entries, lr_exits, _, vol_entries = signal_calculations(lr, lr_ma, lr_mstd, volume, vol_mstd, lr_thld, vol_thld)
    final_entries = lr_entries & vol_entries
    return final_entries, lr_exits

ENTRY_SIGNALS = vbt.IndicatorFactory(
    input_names=['lr', 'volume'],
    param_names=['lr_thld', 'vol_thld', 'lag'],
    output_names=['entries','exits']
).from_apply_func(signals_nb, use_ray=True)
lr_thld = -np.linspace(0,5, 30, endpoint=True)
vol_thld = np.linspace(0,10, 30, endpoint=True)
lag = list(range(5,50))
signals = ENTRY_SIGNALS.run(lr_ind.lr, volume, lr_thld, vol_thld, lag,
                            param_product=True, short_name="signals")
signals.entries.head()

In [ ]:
entries = np.where(signals.entries == True, 1, np.nan)
all_signals = pd.DataFrame(np.where(signals.exits == True, -1, entries), columns=signals.entries.columns)

In [ ]:
@njit
def k_mean(col, arr, *args):
    indexes = np.where(np.isfinite(arr))[0]
    lr = args[0]
    n = 1
    adder = 0
    counter = 0
    while n < len(indexes):
        i = indexes[n]
        prev = indexes[n-1]
        if arr[prev] == 1 and arr[i] == -1:
            adder += lr[i] - lr[prev]
            counter += 1
        n +=1
    return adder / counter if counter != 0 else 0
entry_exit_lr_avg = all_signals.vbt.reduce(k_mean, lr_ind.lr.to_numpy())
entry_exit_lr_avg.vbt.volume().write_html(f"{module_path}/ADA-mean-optimization.html")

In [ ]:
#volume.write_html(f"{module_path}/lag-thlds-optimization.html")



In [ ]:
@njit
def positive_return_prob(col, arr, *args):
    indexes = np.where(np.isfinite(arr))[0]
    lr = args[0]
    n = 1
    adder = 0
    counter = 0
    while n < len(indexes):
        i = indexes[n]
        prev = indexes[n-1]
        #if entry then exit
        if arr[prev] == 1 and arr[i] == -1:
            # if positive return
            if lr[i] > lr[prev]:
                adder += 1
            counter += 1
        n +=1
    return adder / counter if counter != 0 else 0
entry_exit_prob_avg = all_signals.vbt.reduce(positive_return_prob, lr_ind.lr.to_numpy())
entry_exit_prob_avg.vbt.volume().write_html(f"{module_path}/ADA-return_prob-optimization.html")

In [ ]:
@njit
def trades_count(col, arr):
    indexes = np.where(np.isfinite(arr))[0]
    n = 1
    counter = 0
    while n < len(indexes):
        i = indexes[n]
        prev = indexes[n-1]
        if arr[prev] == 1 and arr[i] == -1:
            counter += 1
        n +=1
    return counter
entry_exit_trades_count = all_signals.vbt.reduce(trades_count)
entry_exit_trades_count.vbt.volume().write_html(f"{module_path}/ADA-trade_count-optimization.html")

In [ ]:
best_indexes = entry_exit_prob_avg.nlargest(20, keep="all")

In [ ]:
# graficamos con el mejor de los resultados de la optimización
best_idx = best_indexes[-1]
best_lr_thld, best_vol_thld, best_lag = best_idx

In [ ]:
# recalculamos algunas cosas para graficar y entender lo que está pasando
ups, downs, lr_entries, lr_exits, vol_thld_std, vol_entries = signal_calculations(
    lr_ind.lr.to_numpy(),
    ma_mstd.lr_ma[best_lag].to_numpy(),
    ma_mstd.lr_mstd[best_lag].to_numpy(),
    volume.to_numpy(),
    ma_mstd.vol_mstd[best_lag].to_numpy(),
    best_lr_thld,
    best_vol_thld)

In [ ]:
def plot_series_vs_scatters(series_list: list, booleans_list):
    index = None
    series = series_list.pop(0)
    fig = series.vbt.plot()
    while len(series_list):
        series = series_list.pop(0)
        if not isinstance(series, pd.Series):
            series = pd.Series(series, index=index, copy=True)
        elif index is None:
            index = series.index
        series.vbt.plot(fig=fig)
    i = 1
    for scatter in booleans_list:
        if not isinstance(scatter, pd.Series):
            scatter = pd.Series(scatter, index=index, copy=True)
        elif index is None:
            index = series.index
        scatter = where_true_set_series(series, scatter)
        scatter.name = i
        i += 1
        fig = scatter.vbt.scatterplot(fig=fig)
    return fig

entries = signals.entries[best_idx]
plot_series_vs_scatters([lr_ind.lr, downs], []).show()

In [ ]:
plots = [
    plot_series_vs_scatters([lr_ind.lr, downs.T], []) #, downs, ups], [lr_entries, lr_exits]),
    #plot_series_vs_scatters([volume, vol_thld_std], [vol_entries]),
    #(entries.where(entries == True, np.nan)).vbt.scatterplot()
]

In [ ]:
def add_all_subplots(fig, row, col, list):
    for a in list:
        fig.add_trace(a, row=row, col=col)

fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    vertical_spacing=0.02)
for i in range(len(plots)):
    add_all_subplots(fig, i+1, 1, plots[i].data)

fig.update_layout(height=700, legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

In [ ]:
@njit
def double_multiplier_nb(values, x, y):
    return values*x, values*y

DOUBLE_MULTIPLIER = vbt.IndicatorFactory(
    input_names=['values'],
    param_names=['x', 'y'],
    output_names=['x_mu','y_mu']
).from_apply_func(double_multiplier_nb)
x = np.linspace(0,2,5, endpoint=True)
y = -np.linspace(0,2,5, endpoint=True)
tp_sl = DOUBLE_MULTIPLIER.run(mstd_ind.mstd, x, y, param_product=True, short_name="tp_sl")

In [ ]:
tp_exits = lr_ind.lr_above(tp_sl.x_mu)
sl_exits = lr_ind.lr_below(tp_sl.y_mu)
final_exits = tp_exits.vbt | sl_exits.vbt
final_exits.columns = final_exits.columns.rename("lag", level=-1)
final_exits.head()